In [1]:
import pandas as pd
import numpy as np
import datetime
import glob

#### **進站資料**

In [2]:
files_in = glob.glob('MRTIN_*.csv')
df = []
for f in files_in:
    df.append(pd.read_csv(f, encoding = 'big5', dtype = 'str'))
df_in = pd.concat(df, ignore_index=True)
df_in = df_in.dropna(how = 'all')
mrt_in = df_in.columns.tolist()[2:]
df_in[mrt_in] = df_in[mrt_in].apply(lambda x:x.str.replace(',','')).astype(float)
df_in['星期'] = pd.to_datetime(df_in['日期']).dt.dayofweek+1
# df_in['星期'] = df_in['日期'].dt.dayofweek+1
df_in['月份'] = pd.to_datetime(df_in['日期']).apply(lambda x:x.month)
del df_in['日期']
df2_in = df_in.groupby(['月份', '星期', '時段']).sum()
df2_in = df2_in.reset_index(level=0)
df2_in.index = 'in ' + df2_in.index.map(str)

#### **出站資料**

In [3]:
files_out = glob.glob('MRTOUT_*.csv')
df = []
for f in files_out:
    df.append(pd.read_csv(f, encoding = 'big5', dtype = 'str'))
df_out = pd.concat(df, ignore_index=True)
df_out = df_out.dropna(how = 'all')
mrt_out = df_out.columns.tolist()[2:]
df_out[mrt_out] = df_out[mrt_out].apply(lambda x:x.str.replace(',','')).astype(float)
df_out['星期'] = pd.to_datetime(df_out['日期']).dt.dayofweek+1
# df_out['星期'] = df_out['日期'].dt.dayofweek+1
df_out['月份'] = pd.to_datetime(df_out['日期']).apply(lambda x:x.month)
del df_out['日期']
df2_out = df_out.groupby(['月份', '星期', '時段']).sum()
df2_out = df2_out.reset_index(level=0)
df2_out.index = 'out ' + df2_out.index.map(str)

#### **合併**

In [4]:
df_c = pd.concat([df2_in, df2_out])
for i in range(1,13):
    out_all = 'MRTALL_2016_' + str(i) +'.csv'
    df_all = df_c.loc[df_c['月份'] == i, :]
    del df_all['月份']
    df_all = df_all.T
    df_all.to_csv(out_all, encoding = 'big5')